# Simulate R-map with various sample size and (combinations of) circular/clocklike

In [1]:
from src.erdbeermet.simulation import simulate
import os

In [3]:
numRep = 25000
for n in [6, 10, 15, 20, 25]:
    for i in range(1, 1+numRep):
        
        if not os.path.isdir(f'./simulations/N{n}/default'):
            os.mkdir(f'./simulations/N{n}/default')

        if not os.path.isdir(f'./simulations/N{n}/circ'):
            os.mkdir(f'./simulations/N{n}/circ')
        
        if not os.path.isdir(f'./simulations/N{n}/clock'):
            os.mkdir(f'./simulations/N{n}/clock')

        if not os.path.isdir(f'./simulations/N{n}/circ_clock'):
            os.mkdir(f'./simulations/N{n}/circ_clock')

        scenario = simulate(n, branching_prob=0.0, circular=False, clocklike=False)
        scenario.write_history(f'./simulations/N{n}/default/hist_rep{i}.txt')

        scenario = simulate(n, branching_prob=0.0, circular=True, clocklike=False)
        scenario.write_history(f'./simulations/N{n}/circ/hist_rep{i}.txt')

        scenario = simulate(n, branching_prob=0.0, circular=False, clocklike=True)
        scenario.write_history(f'./simulations/N{n}/clock/hist_rep{i}.txt')

        scenario = simulate(n, branching_prob=0.0, circular=True, clocklike=True)
        scenario.write_history(f'./simulations/N{n}/circ_clock/hist_rep{i}.txt')




In [2]:
import multiprocessing as mp
def multi_run(fun, prms, processes = 4, output=False):
    """Implementation of running in Parallel.
    fun: Function
    prms: The Parameter Files
    processes: How many Processes to use"""
    if output:
        print(f"Running {len(prms)} total jobs; {processes} in parallel.")
    
    if len(prms)>1:
        if output:
            print("Starting Pool of multiple workers...")    
        with mp.Pool(processes = processes) as pool:
            results = pool.starmap(fun, prms)
    elif len(prms)==1:
        if output:
            print("Running single process...")
        results = fun(*prms[0])
    else:
        raise RuntimeWarning("Nothing to run! Please check input.")
    return results

In [2]:
from src.erdbeermet.simulation import load
from erdbeermet.recognition import recognize, recognize_and_compare
from tqdm import tqdm

# prms = [[load(f'./simulations/N20/default/hist_rep{r}.txt').D, True, False] for r in range(1,51)]
# multi_run(recognize, prms, processes=4)

for r in tqdm(range(1,2)):
    #print(f'running replicate {r}')
    scenario = load(f'./simulations/N10/default/hist_rep{r}.txt')
    #recognition_tree = recognize(scenario.D, first_candidate_only=True, print_info=False)

    recognition_tree = recognize_and_compare(scenario, first_candidate_only=True, print_info=False)




100%|██████████| 1/1 [00:00<00:00,  3.39it/s]

[(0, 7, 9, 0.026595445448206634), (5, 6, 8, 0.9977359376105442), (5, 6, 7, 0.3541694896073611), (3, 5, 6, 0.774516890800896), (0, 1, 5, 0.0015728473472632826), (0, 2, 1, 0.8852570015608169)]
[(0, 0: 1) 1.000, (0, 1: 2) 0.645, (2, 0: 3) 0.854, (2, 3: 4) 0.039, (1, 0: 5) 0.998, (3, 5: 6) 0.775, (6, 5: 7) 0.646, (5, 6: 8) 0.998, (0, 7: 9) 0.027]
number of common triples: 0
final quadruple: [0, 2, 3, 4]
init_quad_recovered: False
